# PDP Plots Comparison

This notebook compares the new `pdp_sk` and `pdp_sm` functions with the existing `pred_plot_sk` and `pred_plot_sm` functions across different model types.

Key differences:
- `pred_plot_*` (fast mode): Sets other variables to mean/mode, varies one variable
- `pdp_*` with `mode="pdp"`: True partial dependence - samples from data distribution and averages predictions

## Models covered:
1. Random Forest (classification & regression)
2. XGBoost (classification & regression)
3. MLP Neural Network (classification & regression)
4. Logistic Regression (pyrsm.logistic)
5. Linear Regression (pyrsm.regress)

In [1]:
import time
import polars as pl
import pyrsm as rsm
from pyrsm.model.rforest import rforest
from pyrsm.model.xgboost import xgboost
from pyrsm.model.mlp import mlp
from pyrsm.model.logistic import logistic
from pyrsm.model.regress import regress
from pyrsm.model.visualize import pdp_sk, pdp_sm, pred_plot_sk, pred_plot_sm

In [2]:
## setup pyrsm for autoreload
%reload_ext autoreload
%autoreload 2
%aimport pyrsm

## Load Data

In [3]:
# Classification data
titanic = pl.read_parquet("https://github.com/radiant-ai-hub/raw/refs/heads/main/examples/data/model/titanic.parquet")


In [4]:
rsm.md("https://raw.githubusercontent.com/radiant-ai-hub/refs/heads/main/examples/data/model/titanic_description.md")


## Titanic

This dataset describes the survival status of individual passengers on the Titanic. The titanic data frame does not contain information from the crew, but it does contain actual ages of (some of) the passengers. The principal source for data about Titanic passengers is the Encyclopedia Titanica. One of the original sources is Eaton & Haas (1994) Titanic: Triumph and Tragedy, Patrick Stephens Ltd, which includes a passenger list created by many researchers and edited by Michael A. Findlay.

## Variables

* survival - Survival (Yes, No)
* pclass - Passenger Class (1st, 2nd, 3rd)
* sex - Sex (female, male)
* age - Age in years
* sibsp - Number of Siblings/Spouses Aboard
* parch - Number of Parents/Children Aboard
* fare - Passenger Fare
* name - Name
* cabin - Cabin
* embarked - Port of Embarkation (Cherbourg, Queenstown, Southampton)

##  Notes

`pclass` is a proxy for socio-economic status (SES) 1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1). If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch) some relations were ignored.  The following are the definitions used for sibsp and parch.

Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent:   Mother or Father of Passenger Aboard Titanic
Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins, nephews/nieces, aunts/uncles, and in-laws. Some children travelled only with a nanny, therefore parch=0 for them.  As well, some travelled with very close friends or neighbors in a village, however, the definitions do not support such relations.

Note: Missing values and the `ticket` variable were removed from the data

## Related reading

<a href="http://phys.org/news/2012-07-shipwrecks-men-survive.html" target="_blank">In shipwrecks, men more likely to survive</a>

In [5]:
titanic = titanic.drop_nulls(subset=["age"])
print(f"Titanic: {titanic.shape}")
titanic.head()

Titanic: (1043, 10)


pclass,survived,sex,age,sibsp,parch,fare,name,cabin,embarked
enum,enum,enum,f64,i32,i32,f64,str,str,enum
"""1st""","""Yes""","""female""",29.0,0,0,211.337494,"""Allen, Miss. Elisabeth Walton""","""B5""","""Southampton"""
"""1st""","""Yes""","""male""",0.9167,1,2,151.550003,"""Allison, Master. Hudson Trevor""","""C22 C26""","""Southampton"""
"""1st""","""No""","""female""",2.0,1,2,151.550003,"""Allison, Miss. Helen Loraine""","""C22 C26""","""Southampton"""
"""1st""","""No""","""male""",30.0,1,2,151.550003,"""Allison, Mr. Hudson Joshua Cre…","""C22 C26""","""Southampton"""
"""1st""","""No""","""female""",25.0,1,2,151.550003,"""Allison, Mrs. Hudson J C (Bess…","""C22 C26""","""Southampton"""


In [6]:
# Regression data
salary = pl.read_parquet("https://github.com/radiant-ai-hub/raw/refs/heads/main/examples/data/basics/salary.parquet")


In [7]:
rsm.md("https://raw.githubusercontent.com/radiant-ai-hub/refs/heads/main/examples/data/basics/salary_description.md")


## Salaries for Professors

### Description

The 2008-09 nine-month academic salary for Assistant Professors, Associate Professors and Professors in a college in the U.S. The data were collected as part of the on-going effort of the college's administration to monitor salary differences between male and female faculty members. A data frame with 397 observations on the following 6 variables.

### Variables

- rank = a factor with levels AsstProf, AssocProf, and Prof
- discipline = a factor with levels A ('theoretical' departments) or B ('applied' departments)
- yrs_since_phd = years since PhD
- yrs_service = years of service
- sex = a factor with levels Female and Male
- salary = nine-month salary, in dollars

### Source

Fox J. and Weisberg, S. (2011) An R Companion to Applied Regression, Second Edition Sage.


In [8]:
print(f"Salary: {salary.shape}")
salary.head()

Salary: (397, 6)


salary,rank,discipline,yrs_since_phd,yrs_service,sex
i32,enum,enum,i32,i32,enum
139750,"""Prof""","""B""",19,18,"""Male"""
173200,"""Prof""","""B""",20,16,"""Male"""
79750,"""AsstProf""","""B""",4,3,"""Male"""
115000,"""Prof""","""B""",45,39,"""Male"""
141500,"""Prof""","""B""",40,41,"""Male"""


In [9]:
# Common variables
clf_evars = ["age", "sex", "pclass", "sibsp"]
reg_evars = ["yrs_since_phd", "yrs_service", "rank", "discipline"]

---
# 1. Random Forest

## 1a. Random Forest Classification

In [10]:
rf_clf = rforest(
    data=titanic.to_pandas(),
    rvar="survived", lev="Yes", evar=clf_evars,
    mod_type="classification", n_estimators=100, random_state=1234,
)
rf_clf.summary()

Random Forest
Data                 : Not provided
Response variable    : survived
Level                : Yes
Explanatory variables: age, sex, pclass, sibsp
OOB                  : True
Model type           : classification
Nr. of features      : (4, 6)
Nr. of observations  : 1,043
max_features         : sqrt (2)
n_estimators         : 100
min_samples_leaf     : 1
max_samples          : 1.0
random_state         : 1234
AUC                  : 0.812

Estimation data      :
shape: (5, 6)
┌────────┬───────┬──────────┬────────────┬────────────┬────────────┐
│ age    ┆ sibsp ┆ sex_male ┆ pclass_1st ┆ pclass_2nd ┆ pclass_3rd │
│ ---    ┆ ---   ┆ ---      ┆ ---        ┆ ---        ┆ ---        │
│ f64    ┆ i32   ┆ u8       ┆ u8         ┆ u8         ┆ u8         │
╞════════╪═══════╪══════════╪════════════╪════════════╪════════════╡
│ 29.0   ┆ 0     ┆ 0        ┆ 1          ┆ 0          ┆ 0          │
│ 0.9167 ┆ 1     ┆ 1        ┆ 1          ┆ 0          ┆ 0          │
│ 2.0    ┆ 1     ┆ 0        ┆ 

In [11]:
plot_pred = pred_plot_sk(rf_clf.fitted, titanic, incl=["age", "sex", "pclass"], nnv=30)

In [ ]:
# pred_plot_sk (existing)
start = time.time()
plot_pred = pred_plot_sk(rf_clf.fitted, titanic, incl=["age", "sex", "pclass"], nnv=30)
rf_clf_pred_time = time.time() - start
print(f"pred_plot_sk: {rf_clf_pred_time:.3f}s")
plot_pred

In [ ]:
# pdp_sk - Fast Mode
plot_fast, data_fast, rf_clf_fast_time = pdp_sk(
    rf_clf.fitted, titanic, incl=["age", "sex", "pclass"], mode="fast", grid_resolution=30
)
print(f"pdp_sk (fast): {rf_clf_fast_time:.3f}s")
plot_fast

In [ ]:
# pdp_sk - PDP Mode
plot_pdp, data_pdp, rf_clf_pdp_time = pdp_sk(
    rf_clf.fitted, titanic, incl=["age", "sex", "pclass"], mode="pdp", n_sample=500, grid_resolution=30
)
print(f"pdp_sk (pdp): {rf_clf_pdp_time:.3f}s")
plot_pdp

In [ ]:
# Compare underlying data
print("Fast mode - age predictions:")
print(data_fast["age"].head(10))
print("\nPDP mode - age predictions:")
print(data_pdp["age"].head(10))

In [ ]:
# Interaction plots - all types
# num:cat (age:sex, age:pclass), cat:cat (sex:pclass), num:num (age:sibsp)
plot_int, _, rf_clf_int_time = pdp_sk(
    rf_clf.fitted, titanic, incl=[],
    incl_int=["age:sex", "age:pclass", "sex:pclass", "age:sibsp"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"RF Classification Interactions: {rf_clf_int_time:.3f}s")
plot_int

## 1b. Random Forest Regression

In [ ]:
rf_reg_rf = rforest(
    data=salary.to_pandas(),
    rvar="salary", evar=reg_evars,
    mod_type="regression", n_estimators=100, random_state=1234,
)
rf_reg.summary()

In [ ]:
plot_rf_reg, _, rf_reg_pdp_time = pdp_sk(
    rf_reg.fitted, salary, incl=["yrs_since_phd", "yrs_service", "rank"],
    mode="pdp", n_sample=300, grid_resolution=30
)
print(f"pdp_sk regression: {rf_reg_pdp_time:.3f}s")
plot_rf_reg

In [ ]:
# RF Regression Interactions
# num:cat, cat:cat, num:num
plot_rf_reg_int, _, rf_reg_int_time = pdp_sk(
    rf_reg.fitted, salary, incl=[],
    incl_int=["yrs_since_phd:rank", "rank:discipline", "yrs_since_phd:yrs_service"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"RF Regression Interactions: {rf_reg_int_time:.3f}s")
plot_rf_reg_int

---
# 2. XGBoost

## 2a. XGBoost Classification

In [ ]:
xgb_clf = xgboost(
    data=titanic.to_pandas(),
    rvar="survived", lev="Yes", evar=clf_evars,
    mod_type="classification", n_estimators=100, max_depth=4, random_state=1234,
)
xgb_clf.summary()

In [ ]:
# pred_plot_sk
start = time.time()
plot_xgb_pred = pred_plot_sk(xgb_clf.fitted, titanic, incl=["age", "sex", "pclass"], nnv=30)
xgb_clf_pred_time = time.time() - start
print(f"pred_plot_sk: {xgb_clf_pred_time:.3f}s")
plot_xgb_pred

In [ ]:
# pdp_sk - Fast
plot_xgb_fast, _, xgb_clf_fast_time = pdp_sk(
    xgb_clf.fitted, titanic, incl=["age", "sex", "pclass"], mode="fast", grid_resolution=30
)
print(f"pdp_sk (fast): {xgb_clf_fast_time:.3f}s")
plot_xgb_fast

In [ ]:
# pdp_sk - PDP
plot_xgb_pdp, _, xgb_clf_pdp_time = pdp_sk(
    xgb_clf.fitted, titanic, incl=["age", "sex", "pclass"], mode="pdp", n_sample=500, grid_resolution=30
)
print(f"pdp_sk (pdp): {xgb_clf_pdp_time:.3f}s")
plot_xgb_pdp

In [ ]:
# XGBoost Classification Interactions
plot_xgb_clf_int, _, xgb_clf_int_time = pdp_sk(
    xgb_clf.fitted, titanic, incl=[],
    incl_int=["age:sex", "age:pclass", "sex:pclass", "age:sibsp"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"XGBoost Classification Interactions: {xgb_clf_int_time:.3f}s")
plot_xgb_clf_int

## 2b. XGBoost Regression

In [ ]:
xgb_reg = xgboost(
    data=salary.to_pandas(),
    rvar="salary", evar=reg_evars,
    mod_type="regression", n_estimators=100, max_depth=4, random_state=1234,
)
xgb_reg.summary()

In [ ]:
plot_xgb_reg, _, xgb_reg_pdp_time = pdp_sk(
    xgb_reg.fitted, salary, incl=["yrs_since_phd", "yrs_service", "rank"],
    mode="pdp", n_sample=300, grid_resolution=30
)
print(f"pdp_sk regression: {xgb_reg_pdp_time:.3f}s")
plot_xgb_reg

In [ ]:
# XGBoost Regression Interactions
plot_xgb_reg_int, _, xgb_reg_int_time = pdp_sk(
    xgb_reg.fitted, salary, incl=[],
    incl_int=["yrs_since_phd:rank", "rank:discipline", "yrs_since_phd:yrs_service"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"XGBoost Regression Interactions: {xgb_reg_int_time:.3f}s")
plot_xgb_reg_int

---
# 3. MLP Neural Network

## 3a. MLP Classification

In [ ]:
mlp_clf = mlp(
    data=titanic.to_pandas(),
    rvar="survived", lev="Yes", evar=clf_evars,
    mod_type="classification", hidden_layer_sizes=(10, 5), activation="tanh", random_state=1234,
)
mlp_clf.summary()

In [ ]:
# pred_plot_sk
mlp_clf.plot(plots="pred") #, incl=["pclass", "sex"], nnv=30)

In [ ]:
# pred_plot_sk - pass full mlp object for scaling
start = time.time()
plot_mlp_pred = pred_plot_sk(mlp_clf, titanic)
mlp_clf_pred_time = time.time() - start
print(f"pred_plot_sk: {mlp_clf_pred_time:.3f}s")
plot_mlp_pred

In [ ]:
# pdp_sk - Fast - pass full mlp object for scaling
plot_mlp_fast, _, mlp_clf_fast_time = pdp_sk(
    mlp_clf, titanic, incl=["age", "sex", "pclass"], mode="fast", grid_resolution=30
)
print(f"pdp_sk (fast): {mlp_clf_fast_time:.3f}s")
plot_mlp_fast

In [ ]:
# pdp_sk - PDP - pass full mlp object for scaling
plot_mlp_pdp, _, mlp_clf_pdp_time = pdp_sk(
    mlp_clf, titanic, incl=["age", "sex", "pclass"], mode="pdp", n_sample=500, grid_resolution=30
)
print(f"pdp_sk (pdp): {mlp_clf_pdp_time:.3f}s")
plot_mlp_pdp

In [ ]:
# MLP Classification Interactions - pass full mlp object for scaling
plot_mlp_clf_int, _, mlp_clf_int_time = pdp_sk(
    mlp_clf, titanic, incl=[],
    incl_int=["age:sex", "age:pclass", "sex:pclass", "age:sibsp"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"MLP Classification Interactions: {mlp_clf_int_time:.3f}s")
plot_mlp_clf_int

## 3b. MLP Regression

In [ ]:
mlp_reg = mlp(
    data=salary.to_pandas(),
    rvar="salary", evar=reg_evars,
    mod_type="regression", hidden_layer_sizes=(2, 2), activation="tanh", random_state=1234,
)
mlp_reg.summary()

In [ ]:
# pdp_sk - pass full mlp object for scaling
plot_mlp_reg, _, mlp_reg_pdp_time = pdp_sk(
    mlp_reg, salary, incl=["yrs_since_phd", "yrs_service", "rank"],
    mode="pdp", n_sample=300, grid_resolution=30
)
print(f"pdp_sk regression: {mlp_reg_pdp_time:.3f}s")
plot_mlp_reg

In [ ]:
# MLP Regression Interactions - pass full mlp object for scaling
plot_mlp_reg_int, _, mlp_reg_int_time = pdp_sk(
    mlp_reg, salary, incl=[],
    incl_int=["yrs_since_phd:rank", "rank:discipline", "yrs_since_phd:yrs_service"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"MLP Regression Interactions: {mlp_reg_int_time:.3f}s")
plot_mlp_reg_int

# Logistic Regression Interactions - all types
plot_log_int, _, log_int_time = pdp_sm(
    log_model.fitted, titanic, incl=[],
    incl_int=["age:sex", "age:pclass", "sex:pclass", "age:sibsp"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"Logistic Regression Interactions: {log_int_time:.3f}s")
plot_log_int

In [ ]:
log_model = logistic(
    data=titanic,
    rvar="survived", lev="Yes", evar=clf_evars,
)
log_model.summary()

In [ ]:
# pred_plot_sm (existing)
start = time.time()
plot_log_pred = pred_plot_sm(log_model.fitted, titanic, incl=["age", "sex", "pclass"], nnv=30)
log_pred_time = time.time() - start
print(f"pred_plot_sm: {log_pred_time:.3f}s")
plot_log_pred

In [ ]:
# pdp_sm - Fast
plot_log_fast, _, log_fast_time = pdp_sm(
    log_model.fitted, titanic, incl=["age", "sex", "pclass"], mode="fast", grid_resolution=30
)
print(f"pdp_sm (fast): {log_fast_time:.3f}s")
plot_log_fast

In [ ]:
# pdp_sm - PDP
plot_log_pdp, _, log_pdp_time = pdp_sm(
    log_model.fitted, titanic, incl=["age", "sex", "pclass"], mode="pdp", n_sample=500, grid_resolution=30
)
print(f"pdp_sm (pdp): {log_pdp_time:.3f}s")
plot_log_pdp

In [ ]:
# Logistic Regression Interactions - all types
plot_log_int, _, log_int_time = pdp_sm(
    log_model.fitted, titanic, incl=[],
    incl_int=["age:sex", "age:pclass", "sex:pclass", "age:sibsp"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"Logistic Regression Interactions: {log_int_time:.3f}s")
plot_log_int

# OLS Regression Interactions - all types
plot_ols_int, _, ols_int_time = pdp_sm(
    ols_model.fitted, salary, incl=[],
    incl_int=["yrs_since_phd:rank", "rank:discipline", "yrs_since_phd:yrs_service"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"OLS Regression Interactions: {ols_int_time:.3f}s")
plot_ols_int

In [ ]:
ols_model = regress(
    data=salary,
    rvar="salary", evar=reg_evars,
)
ols_model.summary()

In [ ]:
# pred_plot_sm (existing)
start = time.time()
plot_ols_pred = pred_plot_sm(ols_model.fitted, salary, incl=["yrs_since_phd", "yrs_service", "rank"], nnv=30)
ols_pred_time = time.time() - start
print(f"pred_plot_sm: {ols_pred_time:.3f}s")
plot_ols_pred

In [ ]:
# pdp_sm - Fast
plot_ols_fast, _, ols_fast_time = pdp_sm(
    ols_model.fitted, salary, incl=["yrs_since_phd", "yrs_service", "rank"], mode="fast", grid_resolution=30
)
print(f"pdp_sm (fast): {ols_fast_time:.3f}s")
plot_ols_fast

In [ ]:
# pdp_sm - PDP
plot_ols_pdp, _, ols_pdp_time = pdp_sm(
    ols_model.fitted, salary, incl=["yrs_since_phd", "yrs_service", "rank"], mode="pdp", n_sample=300, grid_resolution=30
)
print(f"pdp_sm (pdp): {ols_pdp_time:.3f}s")
plot_ols_pdp

In [ ]:
# OLS Regression Interactions - all types
plot_ols_int, _, ols_int_time = pdp_sm(
    ols_model.fitted, salary, incl=[],
    incl_int=["yrs_since_phd:rank", "rank:discipline", "yrs_since_phd:yrs_service"],
    mode="pdp", n_sample=300, grid_resolution=20
)
print(f"OLS Regression Interactions: {ols_int_time:.3f}s")
plot_ols_int

---
# 6. Timing Summary

In [ ]:
print("=" * 65)
print("TIMING SUMMARY")
print("=" * 65)

print(f"\n--- SKLEARN MODELS (pdp_sk) ---")

print(f"\nRandom Forest Classification (n={titanic.shape[0]}):")
print(f"  pred_plot_sk:     {rf_clf_pred_time:.3f}s")
print(f"  pdp_sk (fast):    {rf_clf_fast_time:.3f}s")
print(f"  pdp_sk (pdp):     {rf_clf_pdp_time:.3f}s")

print(f"\nXGBoost Classification:")
print(f"  pred_plot_sk:     {xgb_clf_pred_time:.3f}s")
print(f"  pdp_sk (fast):    {xgb_clf_fast_time:.3f}s")
print(f"  pdp_sk (pdp):     {xgb_clf_pdp_time:.3f}s")

print(f"\nMLP Classification:")
print(f"  pred_plot_sk:     {mlp_clf_pred_time:.3f}s")
print(f"  pdp_sk (fast):    {mlp_clf_fast_time:.3f}s")
print(f"  pdp_sk (pdp):     {mlp_clf_pdp_time:.3f}s")

print(f"\n--- STATSMODELS (pdp_sm) ---")

print(f"\nLogistic Regression (n={titanic.shape[0]}):")
print(f"  pred_plot_sm:     {log_pred_time:.3f}s")
print(f"  pdp_sm (fast):    {log_fast_time:.3f}s")
print(f"  pdp_sm (pdp):     {log_pdp_time:.3f}s")

print(f"\nOLS Regression (n={salary.shape[0]}):")
print(f"  pred_plot_sm:     {ols_pred_time:.3f}s")
print(f"  pdp_sm (fast):    {ols_fast_time:.3f}s")
print(f"  pdp_sm (pdp):     {ols_pdp_time:.3f}s")

---
# Notes

- **Fast mode** is similar to `pred_plot_*` - sets other variables to mean/mode
- **PDP mode** averages predictions across the data distribution (true partial dependence)
- PDP mode is slower but gives more accurate marginal effects
- Use `n_sample` to control speed vs accuracy tradeoff in PDP mode
- Both functions return `(plot, data_dict, runtime)` for inspection and testing
- `pdp_sk` works with sklearn models (rforest, xgboost, mlp)
- `pdp_sm` works with statsmodels models (logistic, regress)